__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x28_Templates_überprüfen.ipynb)__

# Templates überprüfen
* dient nur zur Kontrolle, ist nicht notwendig zur Erstellung des Wörterbuches
* erstellt die Datei wiktionary_templates.pickle
* parst dafür wikitext mit mwparserfromhell

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 17:51:44


time: 409 ms


In [2]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, swifter

except ImportError as e:
    !pip install pandasklar swifter
    import pandasklar as pak, bpyth as bpy, swifter
    
grid       = pak.grid
check_mask = pak.check_mask      

time: 4.2 s


## Einstellungen

In [2]:
# Einstellungen

version = '_all'
#version = '_short'

# Load
verzeichnis_load                      = 'data_00'
wiktionary_filename                   = verzeichnis_load + '/wiktionary'              + version + '.pickle'  
wiktionary_merkmal_filename           = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename      = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  


# Save
verzeichnis_save                      = 'data_01'
wiktionary_templates_filename         = verzeichnis_save + '/wiktionary_templates'    + version + '.pickle'    

# Was debuggen?
suche_debug = ['Haus',]

time: 11.6 ms


## Vorbereiten

In [4]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 44.4 ms


In [5]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

time: 38.9 ms


In [6]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

time: 43.8 ms


In [4]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen   import *

# mwparserfromhell plus eigene Ergänzungen 
import mwparserfromhell
from x24_process_wikicode                import *
from x22_my_node                         import *

## Funktionen

In [5]:
def parse_templatenames(zeile):
    '''
    Liefert alle verwendeten Templates eines Wiktionary-Abschnittes
    sowie die Länge der Parameter jedes Templates.
    '''
    
    # mwparserfromhell
    wikicode = mwparserfromhell.parse(zeile.wikitext)     
    
    templates = wikicode.filter_templates(recursive=False)  
    templatenames = list()
    for t in templates:  
        name = str(t.name.strip())
        lenp = str(len(t.params))
        if len(name) <= 2:
            continue
        if (len(name) == 3)  and  (name == str.lower(name)):
            continue            
        templatenames.append(name + '☉' + lenp)
    
    result = { 'section_id':    zeile.section_id,
               'section_id2':   zeile.section_id2,
               'templatenames': templatenames
             }

        
    return pd.DataFrame([result])

time: 33.2 ms


In [62]:
# häufigkeit             beschreibt, wie häufig ein Template insgesamt verwendet wird
# count                  beschreibt, wie oft ein Template pro Section verwendet wird.
# lenp_min und lenp_max  beschreiben die Anzahl der Parameter, die bei diesem Template im Mittel verwendet werden.
#                        Wenn ein Template pro Section mehrfach verwendet wird, können sich lenp_min und lenp_max unterscheiden.
#
def show_stats( wiktionary_templates, filter_templates, match_substring=False):
    '''
    Zeigt gruppierte Daten zur Verwendung der Templates im Wiktionary.
    * wiktionary_templates: die gleichnamige Tabelle
    * filter_templates:     Nur Templates, die in dieser Liste sind werden ausgewertet
    * match_substring:      Reicht in filter_templates der Anfang des Namens?
    
    Liefert folgende Daten für jedes Template:
    * lenp_min:     mittleres Minimum der Parameterlänge
    * lenp_max:     mittleres Maximum der Parameterlänge
    * count:        mittlere Häufigkeit pro Abschnitt  
    * section_id2:  Einige Beispiele, wo das Template vorkommt
    * häufigkeit:   Gesamthäufigkeit des Templates im ganzen Wiktionary
    '''
    filter_templates = tuple(filter_templates)
    spalten = ['section_id2','template','lenp_min', 'lenp_max', 'count' ]
    if match_substring:
        mask = wiktionary_templates.template.str.startswith(filter_templates)
    else:
        mask = wiktionary_templates.template.isin(filter_templates)
    df = pak.reset_index(wiktionary_templates[mask][spalten])
    
    gruppiert = df.groupby('template')
    
    result = gruppiert.agg(
      
        lenp_min    = ('lenp_min',    'mean'),   
        lenp_max    = ('lenp_max',    'mean'),  
        count       = ('count',       'mean'),            
        section_id2 = ('section_id2', pak.top_values_20),                 
        häufigkeit  = ('section_id2', 'count')                        

    ).reset_index()
 
    return result.sort_values('häufigkeit', ascending=False)

time: 34.4 ms


## Laden

In [7]:
# Laden
wiktionary = bpy.laden(wiktionary_filename)

time: 10.2 s


In [8]:
try:
    wiktionary_templates = bpy.laden(wiktionary_templates_filename)  # Falls vorhanden
except:
    pass

time: 502 ms


In [9]:
#grid(wiktionary_templates,sample)

time: 39.5 ms


In [10]:
print('Einträge:', wiktionary.shape[0])

Einträge: 806916
time: 39.6 ms


In [11]:
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary, mask)

2 rows out of 806916


time: 247 ms


In [12]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",2.1 GB
1,wiktionary_templates,"(DataFrame, Series)",459.2 MB
2,mask,"(Series, bool)",6.9 MB
3,translate_tag,"(DataFrame, Series)",37.1 KB
4,translate_tagZ,"(DataFrame, Series)",20.9 KB
5,_i6,"(str,)",4.0 KB
6,_i5,"(str,)",2.5 KB
7,Stop,"(type,)",1.0 KB
8,Markdown,"(type,)",1.0 KB
9,HTML,"(type,)",1.0 KB


time: 1.81 s


## parse_templatenames ausprobieren

In [13]:
?parse_templatenames

time: 108 ms


Signature: parse_templatenames(zeile)
Docstring:
Liefert alle verwendeten Templates eines Wiktionary-Abschnittes
sowie die Länge der Parameter jedes Templates.
File:      /tmp/ipykernel_113877/1105431938.py
Type:      function


In [14]:
# Eine Testzeile aus wiktionary (als Series)
mask = wiktionary.lemma == 'Haus'
testzeile = wiktionary[mask].iloc[0]
#testzeile

time: 159 ms


In [15]:
# parse_templatenames ausprobieren
df = parse_templatenames(testzeile)
df

,section_id,section_id2,templatenames
0,21600,Haus,"[Deutsch Substantiv Übersicht☉38, Worttrennung..."


time: 206 ms


In [16]:
df.iloc[0].templatenames[:10]

['Deutsch Substantiv Übersicht☉38',
 'Worttrennung☉0',
 'Pl.☉0',
 'Aussprache☉0',
 'IPA☉0',
 'Lautschrift☉1',
 'Hörbeispiele☉0',
 'Audio☉1',
 'Audio☉1',
 'Reime☉0']

time: 48.4 ms


## wiktionary_templates neu erstellen

In [17]:
# all: 15min
# short: 1min 30 sec
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_templates = pd.concat(wiktionary.swifter.apply(parse_templatenames, axis=1).tolist()  )

time: 19min 23s


In [18]:
# Alle Templates einzeln
wiktionary_templates = wiktionary_templates.explode('templatenames')

# split
wiktionary_templates[['template','lenp']] = wiktionary_templates['templatenames'].str.split('☉', n=1, expand=True)

time: 21.1 s


In [19]:
# lenp
wiktionary_templates.lenp = wiktionary_templates.lenp.astype(int)

time: 668 ms


In [20]:
# Spalten pflegen
wiktionary_templates = pak.drop_cols(wiktionary_templates,['templatenames','section_id'])

time: 3.51 s


In [21]:
mask = wiktionary_templates.section_id2 == 'Haus'
wiktionary_templates[mask]

,section_id2,template,lenp
0,Haus,Deutsch Substantiv Übersicht,38
0,Haus,Worttrennung,0
0,Haus,Pl.,0
0,Haus,Aussprache,0
0,Haus,IPA,0
...,...,...,...
0,Haus,Ref-wissen.de,1
0,Haus,Ref-wissen.de,1
0,Haus,Ref-UniLeipzig,0
0,Haus,Ref-Grimm,0


time: 591 ms


In [22]:
# Manche Templates kommen mehrfach pro Abschnitt vor, daher wird gruppiert

# 7sec
gruppiert = wiktionary_templates.groupby(['section_id2','template'])

wiktionary_templates = gruppiert.agg(

    count      = ('lenp', 'count'),         
    lenp_min   = ('lenp', 'min'),  
    lenp_max   = ('lenp', 'max'),       
            
).reset_index()

time: 8.47 s


In [23]:
grid(wiktionary_templates, pak.sample)

8 rows out of 8892381


time: 12.3 s


In [24]:
# Sparsame Datentypen
wiktionary_templates = pak.change_datatype(wiktionary_templates)
#pak.analyse_cols(wiktionary_templates)

time: 14.9 s


## wiktionary_templates belöschen

In [25]:
# Speicher freimachen
#if 'all' in version:
#    del wiktionary

time: 35.6 ms


In [26]:
# Uninteressante Templates
templates0 =  ['Ü','K','Worttrennung','Aussprache','Sprache','Anmerkung','Reime'] 
templates0 += ['Übersetzungen','Ü-Tabelle','Referenzen','Quellen','IPA','Audio','Lautschrift','Hörbeispiele'] 
templates0 += ['ugs.','va.','landsch.','übertr.','abw.','trans.','intrans.','geh.','scherzh.','refl.','fachspr.','fam.']
templates0 += ['südd.','nordd.','österr.','schweiz.','vatd.','gmh.','goh.','lat.','ahd.','Üxx4','Hebr','CH&LI','fro.']
templates0 += ['mhd.','grc.','Polytonisch','Lautschrift?','Wort des Jahres']
templates0 += ['Sg.','Sg.1','Sg.2','Pl.','Pl.1','Pl.2','Pl.3','kPl.','Komp.','Sup.','Prät.','Part.','Gen.','Dat.','Akk.',]
templates0 += ['Beispiele fehlen','Literatur','Referenzen prüfen','Absatz']
templates0 += ['Wikipedia','Wikiquote','Wikipedia-Suche','Wikisource','Commons','Wikisource-Suche']
templates0 += ['Anmerkungen','Per-Deutsche Welle','Abschnitte fehlen','Dativ-e','Bekannte Namensträger','Fremdaffixe (Deutsch)']
templates0 += ['Internetquelle','Anmerkung scht','erweitern','überarbeiten','Wörter des Jahres']

time: 33.8 ms


In [27]:
?show_stats

time: 59.5 ms


Signature: show_stats(wiktionary_templates, filter_templates, match_substring=False)
Docstring:
Zeigt gruppierte Daten zur Verwendung der Templates im Wiktionary.
* wiktionary_templates: die gleichnamige Tabelle
* filter_templates:     Nur Templates, die in dieser Liste sind werden ausgewertet
* match_substring:      Reicht in filter_templates der Anfang des Namens?

Liefert folgende Daten für jedes Template:
* lenp_min:     mittleres Minimum der Parameterlänge
* lenp_max:     mittleres Maximum der Parameterlänge
* count:        mittlere Häufigkeit pro Abschnitt  
* section_id2:  Einige Beispiele, wo das Template vorkommt
* häufigkeit:   Gesamthäufigkeit des Templates im ganzen Wiktionary
File:      /tmp/ipykernel_113877/68428.py
Type:      function


In [28]:
# Uninteressante Templates
df = show_stats(wiktionary_templates, templates0)
grid(df)

76 rows


time: 11.1 s


In [29]:
# belöschen
mask = wiktionary_templates.template.isin(templates0)
wiktionary_templates = pak.drop_rows(wiktionary_templates, mask)

time: 1.36 s


In [30]:
# reset_index
wiktionary_templates = pak.reset_index(wiktionary_templates)

time: 402 ms


In [31]:
grid(wiktionary_templates, pak.sample)

9 rows out of 3112198


time: 6.4 s


## Speichern

In [32]:
# Speichern
if wiktionary_templates_filename:
    bpy.speichern(wiktionary_templates, wiktionary_templates_filename) 

time: 1.31 s


In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [33]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",2.8 GB
1,wiktionary_templates,"(DataFrame, Series)",475.3 MB
2,mask,"(Series, bool)",8.5 MB
3,testzeile,"(Series, int32)",344.8 KB
4,translate_tag,"(DataFrame, Series)",37.1 KB
5,translate_tagZ,"(DataFrame, Series)",20.9 KB
6,df,"(DataFrame, Series)",14.4 KB
7,_21,"(DataFrame, Series)",9.4 KB
8,_i6,"(str,)",4.0 KB
9,_i5,"(str,)",2.5 KB


time: 47.3 s


## Pläne überprüfen

In [34]:
blab.render_doc(plan_merkmal_erstellen)

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

time: 34.4 ms


In [35]:
# Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal)

102 rows


time: 108 ms


In [36]:
# Alle Extraktionsmethoden
list(plan_merkmal.collect.unique())

['E', 'P', 'L', 'Ü', 'T', '']

time: 39 ms


In [37]:
# Templates, deren Text eingesammelt wird
mask = (plan_merkmal.collect == 'T')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

6 rows


time: 1.2 s


In [38]:
# Templates, deren Links eingesammelt werden
mask = (plan_merkmal.collect == 'L')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

25 rows


time: 3.09 s


In [39]:
# Templates, deren Übersetzungen eingesammelt werden
mask = (plan_merkmal.collect == 'Ü')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

1 rows


time: 557 ms


In [40]:
# Templates, deren Parameter eingesammelt werden
mask = (plan_merkmal.collect == 'P')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template) )
grid(df)

2 rows


time: 304 ms


In [41]:
# Templates, deren Existenz gecheckt wird
mask = (plan_merkmal.collect == 'E')
df = show_stats(wiktionary_templates, list(plan_merkmal[mask].template)).sort_values('häufigkeit', ascending=False)
grid(df)

9 rows


time: 280 ms


## Templates finden, die noch ausgewertet werden könnten

In [42]:
# Templates, die eine Gruppe mit gemeinsamen Oberbegriff kennzeichnen
templates1 =  ['Monate (Deutsch)','Wochentage (Deutsch)','Staaten Europa (Deutsch)','Verwandte (Deutsch)','Bundesstaaten der USA']
templates1 += ['Elemente','Periodensystem','Zahlensystem (Deutsch)','Planeten','Staaten Afrika (Deutsch)','Staaten Asien (Deutsch)']

# Templates, die noch nicht ausgelesen werden / werden können
templates2 =  ['Symbole','Reim'] # 'Wortart'
    
# Templates, die derzeit nicht interessante Informationen liefern
templates3 =  ['Entlehnungen']

known_templates = templates0 + templates1 + templates2 + templates3 + list(plan_merkmal.template)
#known_templates

time: 37.6 ms


In [43]:
# show_stats (templates1 .. templates4 manuell durchschauen!)
df = show_stats(wiktionary_templates, templates3 )
grid(df)

1 rows


time: 183 ms


In [63]:
# restliche
df = pak.analyse_freqs( wiktionary_templates, 'template')
mask1 = ~df.template.isin(known_templates)   
mask2 = ~df.template.str.startswith(  ('Grundformverweis','Ref-','Lit-','QS','Ähnlichkeiten')  )
mask3 = ~df.template.str.endswith(    ('Übersicht','Übersicht m','Übersicht f','Übersicht -sch')  )
restliche = list(df[mask1  &  mask2  &  mask3 ].template)
#restliche

time: 1.06 s


In [64]:
# Kandidaten für Templates, die man noch auswerten könnte
# show_stats 
df = show_stats(wiktionary_templates, restliche ).sort_values('häufigkeit', ascending=False)
bgc('gold')
grid(df)

197 rows


time: 417 ms


## Probeweises extract für ungenutzte Templates

In [46]:
# Parameter
templatename     = 'Reim'
collectmethode   = 'P'

time: 60.2 ms


In [47]:
# Section finden, welches ein bestimmtes Template enthalten
mask = wiktionary_templates.template == templatename
df = wiktionary_templates[mask].sort_values('count',ascending=False)
beispiellemma = df.iloc[1].section_id2
print('beispiellemma =', beispiellemma)
df.head(3)

beispiellemma = zornigsten


,section_id2,template,count,lenp_min,lenp_max
317619,Enquete,Reim,5,2,2
3023550,zornigsten,Reim,4,2,2
2024780,genächtigt,Reim,4,2,2


time: 420 ms


In [48]:
# plan für Testzwecke
plan = pak.dataframe({'template':templatename, 'collect':collectmethode,'merkmal':'TEST'})
plan

,template,collect,merkmal
0,Reim,P,TEST


time: 61.4 ms


In [49]:
mask = (wiktionary.section_id2 == beispiellemma)
testzeile = wiktionary[mask].iloc[0]
#testzeile

wikicode = mwparserfromhell.parse(testzeile.wikitext)   

df = extract_all(testzeile, plan, step=None )
df

,node_debug,node_kontext,node_text,section_id,section_id2,merkmal
0,Reim,1,ɔʁnɪçstn̩,32251200,zornigsten,TEST
1,Reim,2,Deutsch,32251200,zornigsten,TEST


time: 169 ms


## wiktionary_merkmal anschauen

In [50]:
# Suche festlegen
suche_debug = ['Haus',]

time: 47.4 ms


In [51]:
# Laden
wiktionary_merkmal      = bpy.laden(wiktionary_merkmal_filename)

time: 3.74 s


In [53]:
# wiktionary_merkmal
mask = wiktionary.title.isin(suche_debug)
suche_section_id2s = wiktionary[mask].section_id2.tolist()
mask = wiktionary_merkmal.section_id2.isin(suche_section_id2s)
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data'], limits=[999,None])
grid(a)

13 rows


time: 467 ms


In [54]:
mask = (wiktionary_merkmal.node_kontext != '')
df = wiktionary_merkmal[mask]
a = pak.analyse_freqs( df, ['node_kontext','node_debug'], limits=[50,None] )
grid(a)

50 rows


time: 3.43 s


## wiktionary_merkmal_text anschauen

In [55]:
# Laden
wiktionary_merkmal_text = bpy.laden(wiktionary_merkmal_text_filename)  # Falls vorhanden

time: 2.63 s


In [56]:
# wiktionary_merkmal_text
mask = wiktionary.title.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal_text.section_id.isin(suche_section_ids)
grid(wiktionary_merkmal_text, mask)

144 rows out of 619794


time: 125 ms
